In [1]:
!pip install anytree

  Using cached anytree-2.8.0-py2.py3-none-any.whl (41 kB)


In [2]:
import torch
import pickle
from pathlib import Path
import os
import pprint
from anytree import Node, RenderTree
import numpy as np

For future iterations of this I might change the scheme to a deep directory based approach.    
Then iterations would be named `1.pickle`, `2.pickle` and be stored in deep directories like:    
`"Experiment_name/config1/config2/config3/config4/1.pickle"`    
This way I could just split the name accross the folder separator and have all the setting values, then the only thing I need is a list that gives me the correct setting-value assignment, which sounds a lot easier.

# Helper functions

In [3]:
def unpickle(filepath : str):
    file_object = open(filepath, "rb")
    results = pickle.load(file_object)
    file_object.close()
    return results

In [4]:
def extract_setting_from_name(name :str, naming_schema : list):
    result = {}
    # Main assumption for the naming schema: parameters are strictly separated by underscores '_'
    for value, setting in zip(name.split("_"), naming_schema):
        result[setting] = value
    return result
    

In [5]:
def load_results(directory, experiment_name, naming_schema=None):
    path = Path(directory)

    dirs = [e for e in path.iterdir() if e.is_dir() and not str(e) == '.ipynb_checkpoints']
    subdirs = {str(path): [e for e in path.iterdir() if e.is_dir()] for path in dirs}
    
    relevant_subdirs = subdirs[os.path.join(directory, experiment_name)]
    relevant_subdirs.sort()
    #result_filename = '*.pickle'
    pickle_dirs = list()
    for subdir in relevant_subdirs:
        pickle_dirs.extend(sorted(subdir.glob("*.pickle")))
    results = []
    if not naming_schema is None:
        all_attributes = [extract_setting_from_name(subdir.name, naming_schema) for subdir in relevant_subdirs]    
        for attributes, pick in zip(all_attributes, pickle_dirs):
            try:
                unpickled_stuff = unpickle(pick)
                results.append({'attributes': attributes, 'results': unpickled_stuff})
            except:
                # Sometimes there were unknown issues with the pickle files, in those instances we re-ran training
                print("Catastrophic failure")
    else:
        for pick in pickle_dirs:
            try:
                unpickled_stuff = unpickle(pick)
                results.append({'results': unpickled_stuff})
            except:
                # Sometimes there were unknown issues with the pickle files, in those instances we re-ran training
                print("Catastrophic failure")
    return results


In [6]:
def deep_dict_get(data : dict, path : str):
    """
    path a slash ('/') separated path down to 
    """
    temp = data.copy()
    for entry in path.split("/"):
        # Catches leading '/' in tree printing
        if entry == '':
            continue
        temp = temp[entry]
    return temp

In [7]:
def compare_values(d1 : dict, d2 : dict, path : str):
    v1 = deep_dict_get(d1, path)
    v2 = deep_dict_get(d2, path)
    # Sanity check 1
    if not type(v1) == type(v2):
        return False
    else:
        # This can potentially cause errors when comparing lists of lists (or Tensors/Arrays)
        return v1 == v2

In [8]:
def create_common_root_list(root : str, values : list):
    return [f"{root}/{val}" for val in values]

# Main functions    
What features do I need?    
- ~~Show a structure tree of the results (i.e. experiment settings and result values)~~~
- ~~Filtering by setting, given a key~~
- ~~Filtering by values, given a key~~
- Create selected statistics (mean/med/std/quartiles/...) for certain values/keys
- Apply a function to certain values/keys and return the results (e.g. Eigendecomposition/Normalization/...)
- ~~Group all entries that share settings/values~~
- 

In [9]:
def generate_tree_structure(data, parent=None):
    # Solved through recursively going deeper into the data structure and then returning the leafs if at the end

    if parent is None and not len(data.keys()) == 1:
        parent = Node("root")
    elif parent is None:
        parent = Node(list(data.keys())[0])
        data = data[list(data.keys())[0]]
    
    # Recursion condition
    # If there are any dictionaries inside then go deeper
    if not any([type(data[entry]) == dict for entry in data]):
        for entry in data:
            Node(entry, parent=parent) 
    else:
        for entry in data:
            if type(data[entry]) == dict:
                branch = Node(entry, parent=parent)
                generate_tree_structure(data[entry], parent=branch)
            else:
                Node(entry, parent=parent)
    return parent
    

In [10]:
def group_by(list_of_data : list, path, value=None):
    """
    value : If None, make subgroups of equal values. 
            Otherwise return a single group where value is matched
    """
    grouped_data = {}

    if not value is None:
        grouped_data[f"{path} = {value}"] = [data for data in list_of_data if deep_dict_get(data, path) == value]
    else:    
        finished_values = list()
        for data in list_of_data:
            value = deep_dict_get(data, path)
            grouped_data[f"{path} = {value}"].append(data)
            #if not value in finished_values:
            #    grouped_data[f"{path} = value"] = [data for data in list_of_data if deep_dict_get(data, path) == value]
            #    finished_values.append(value)
    return grouped_data

In [11]:
def dict_compare(d1, d2):
    # Checking for empty list in d2, i.e. initial value
    if not type(d1) == type(d2):
        return False
    return all((d1.get(k) == v for k, v in d2.items()))
        

# Could contain an alternative head as (list_of_data : list, paths : [list, dict], values : None) 
# where paths-values would require a 1-to-1 correspondence. 
# But this could quickly become error prone on the user side...
def group_by_multiple(list_of_data : list, paths):
    grouped_data = {}
    # Grouping without values
    if type(paths) == list:
        finished_values = list()
        for data in list_of_data:
            paths_vals = {path : deep_dict_get(data, path) for path in paths}
            # not any X <=> all not X
            # i.e. only succeeds when this combination didn't exist before
            if not any([dict_comapare(paths_vals, fin_val) for fin_val in finished_values]):
                grouped_data[" ; ".join([f"{path} = {paths_vals[path]}" for path in paths_vals])] = [data]
            else:
                grouped_data[" ; ".join([f"{path} = {paths_vals[path]}" for path in paths_vals])].append(data)

    # Grouping by path-value combinations
    # Only returns the group where all those pairs are true
    elif type(paths) == dict:
        # The keys will contain the paths
        # The values will be the corresponding expected values
        # Yes, this could be a one-liner with a very neat nested list creation, 
        #  but I chose readability with temporary variables over it.
        good_data = list()
        for data in list_of_data:
            if all([deep_dict_get(data, path) == paths[path] for path in paths]):
                good_data.append(data)
        grouped_data[" ; ".join([f"{path} = {paths[path]}" for path in paths])] = good_data
    return grouped_data

In [12]:
def filter_by(list_of_data : list, path : str, value):
    filtered_data = list()
    for data in list_of_data:
        if deep_dict_get(data, path) == value:
            filtered_data.append(data)
    return filtered_data

In [13]:
def filter_value(list_of_data : list, path : str):
    """
        returns a list of the target value from each data dict
    """
    list_of_values = list()
    for data in list_of_data:
        list_of_values.append(deep_dict_get(data, path))
    return list_of_values

# Load results

In [16]:
#naming_schema = ["Metric", "Kernel_search", "train_data_ratio", "Data_kernel", "weights", "Variance_list", "eval_START", "eval_END", "eval_COUNT", "optimizer", "train_iterations", "LR", "Noise", "Data_scaling", "BFGS"]
all_results = load_results('results', "hardcoded")
#pprint.pprint(all_results)
result_tree = generate_tree_structure(all_results[0]).descendants
pprint.pprint(result_tree)


# data - kernel assignment : RBF_PER = SIN*RBF; 4PER = 4C*SIN; PER = C*SIN
# Perform for each main dictionary: Iterate over kernels -> Look for the lowest(highest?) loss, 


for main_dict in all_results:
    print("############################")
    print(main_dict["results"]["attributes"]["data_gen"])
    for model_kernel in ['4C*SIN','C*C*RBF','C*RBF','C*SIN','C*SIN + C*SIN','C*SIN + C*SIN + C*SIN','SIN*RBF']:
        print("\n----")
        print(f"{model_kernel}:\n")
        for metric in ["Laplace"]:#, "MC", "MLL", "AIC"]:
            print(f"{metric}\t - \t{main_dict['results'][metric][model_kernel]['loss']}")

(Node('/results/attributes'),
 Node('/results/attributes/eval_START'),
 Node('/results/attributes/eval_END'),
 Node('/results/attributes/eval_COUNT'),
 Node('/results/attributes/optimizer'),
 Node('/results/attributes/train_iters'),
 Node('/results/attributes/LR'),
 Node('/results/attributes/BFGS'),
 Node('/results/attributes/data_gen'),
 Node('/results/Laplace'),
 Node('/results/Laplace/SIN*RBF'),
 Node('/results/Laplace/SIN*RBF/parameter_punishment'),
 Node('/results/Laplace/SIN*RBF/loss'),
 Node('/results/Laplace/SIN*RBF/details'),
 Node('/results/Laplace/SIN*RBF/details/MLL'),
 Node('/results/Laplace/SIN*RBF/details/parameter list'),
 Node('/results/Laplace/SIN*RBF/details/parameter values'),
 Node('/results/Laplace/SIN*RBF/details/corrected Hessian'),
 Node('/results/Laplace/SIN*RBF/details/diag(constructed eigvals)'),
 Node('/results/Laplace/SIN*RBF/details/original symmetrized Hessian'),
 Node('/results/Laplace/SIN*RBF/details/prior mean'),
 Node('/results/Laplace/SIN*RBF/detail

In [18]:
#all_results[0]["results"]["attributes"]["data_gen"]
pprint.pprint(all_results[0]["results"]["Laplace"]["4C*SIN"]["details"])
MLL = all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]["MLL"]
H = all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]["corrected Hessian"]
S = torch.diag(all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]["diag(prior var)"])
t_mu = all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]["prior mean"]
t_0 = all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]["parameter values"]

diff = t_mu - t_0
mid_term = (S.inverse() - H).inverse()
matmuls = diff.t() @ S.inverse() @ mid_term @ H @ diff
print(matmuls)
#print(torch.log(S.det()))
print(MLL - 0.5*torch.log(S.det()) - 0.5*torch.log(mid_term.det()) + 0.5* matmuls)
#pprint.pprint(list(zip(all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]['parameter list'], all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]['parameter values'])))

#pprint.pprint(torch.linalg.eig(all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]["corrected Hessian"]))
#pprint.pprint(torch.linalg.eig(all_results[0]["results"]["Laplace"]["4C*SIN"]["details"]["original symmetrized Hessian"]))

{'Approximation time': 0.0004425048828125,
 'Correction time': 0.0016541481018066406,
 'Derivative time': 0.03189587593078613,
 'MLL': tensor(227.9842, requires_grad=True),
 'Prior generation time': 0.00019431114196777344,
 'Total time': 0.034353017807006836,
 'corrected Hessian': tensor([[-6.2433e-01,  2.6383e-03, -5.9260e-03,  6.1808e-02,  1.1424e-02,
         -4.4135e-03,  6.1808e-02, -2.0227e-03, -3.8999e-03,  6.1808e-02,
          1.9806e-03, -3.8923e-03,  6.1808e-02],
        [ 2.6383e-03, -1.7003e+00,  3.7035e+00, -1.3297e+00, -1.5400e+00,
          2.5947e+00, -6.4673e+00, -4.0572e-01,  3.3184e+00, -4.3488e+00,
         -4.8372e-01,  3.4279e+00,  2.5355e+00],
        [-5.9260e-03,  3.7035e+00, -3.5875e+01,  6.2721e+01,  1.4987e+00,
         -1.5118e+00, -6.9491e+00,  2.4215e+00, -1.1790e+01,  7.2209e+00,
          2.4931e+00, -1.4299e+01, -1.4942e+01],
        [ 6.1808e-02, -1.3297e+00,  6.2721e+01, -7.5123e+07, -8.3576e+00,
         -9.8405e+00, -1.9631e+06, -4.8591e-01,  1.30

In [ ]:
#naming_schema = ["Metric", "Kernel_search", "train_data_ratio", "Data_kernel", "weights", "Variance_list", "eval_START", "eval_END", "eval_COUNT", "optimizer", "train_iterations", "LR", "Noise", "Data_scaling", "BFGS"]
all_results = load_results('results', "Laplace")
print(len(all_results))
result_tree = generate_tree_structure(all_results[0]).descendants
pprint.pprint(result_tree)

print(filter_value(all_results, '/results/results/final model'))

#########
"""
    Looking at the final models that were found and the ratio of exactly the correct one
"""
#########
print("\n########## Laplace ##########\n")
all_final_models = filter_value(all_results, '/results/results/final model')
winners = group_by(all_results, 'results/results/final model', value='(c * PER)')['results/results/final model = (c * PER)']
print(f"Percentage of exactly correct model: {len(winners)/len(all_final_models)*100}")
print(f"Percentage of correct component: {np.sum(['PER' in m for m in all_final_models])}")
print("\n##############\n")
Laplace_runtimes = filter_value(all_results, '/results/results/details/Total time')
#training_runtimes = filter_value(all_results, '/results/results/Training time')
#KS_runtimes = filter_value(all_results, '/results/results/Kernel search time')
print(f"Average Laplace runtime: {np.mean(Laplace_runtimes)}")
#print(f"Average Laplace runtime: {np.mean(KS_runtimes)}")

print("\n#############################\n")

